## Testing CNN implentation with Keras

In [1]:
# Pathing Libraries
from pathlib import Path
import os
import os.path as op
import glob

# Image Libraries
import nibabel as nib
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
## Pathing
path = '../'

images_path = Path(path, 'data', 'images')
masks_path = Path(path, 'data', 'masks')
segs_path = Path(path, 'data', 'segs_refs')
metadata_path = Path(path, 'data', 'meta')

## Functions
def z_normalize(img_data, mask_data):
    # Ensure the mask is binary
    mask_data = np.where(mask_data > 0, 1, 0)

    # Apply the mask to the image
    masked_image_data = img_data * mask_data

    # Calculate the z-score of the masked image
    masked_image_data[mask_data==0] = stats.zscore(masked_image_data[mask_data==0])

    # Handle NaN values that might result from zscoring zero-valued elements
    masked_image_data = np.nan_to_num(masked_image_data)

    return masked_image_data


In [5]:
meta_data = pd.read_csv(Path(metadata_path, 'meta_data_all.csv'))

for subj, age, gender in zip(meta_data['subject_id'], meta_data['age'], meta_data['gender_text']): # ''gender_code'; 1= male, 2= female
    print(subj, age, gender)
    
    # Loading in Raw Image Data
    image = nib.load(op.join(images_path, 'sub-' + subj + '_T1w_unbiased' + '.nii.gz'))
    img_data = image.get_fdata()

    # Loading in Segmentation Data
    seg = nib.load(op.join(segs_path, 'sub-' + subj + '_T1w_seg.nii.gz'))
    seg_data = seg.get_fdata()

    # Loading in Mask Data
    mask = nib.load(op.join(masks_path, 'sub-' + subj + '_T1w_brain_mask' + '.nii.gz'))
    mask_data = mask.get_fdata()

    # Z-score Normalization
    z_img = z_normalize(img_data, mask_data)
    z_seg = z_normalize(seg_data, mask_data)


CC110033 24 MALE
CC110037 18 MALE
CC110045 24 FEMALE
CC110056 22 FEMALE
CC110062 20 MALE
CC110069 28 FEMALE
CC110087 28 FEMALE
CC110098 23 MALE
CC110101 23 MALE
CC110126 22 FEMALE
CC110174 25 FEMALE
CC110182 18 FEMALE
CC110187 25 FEMALE
CC110319 28 FEMALE
CC110411 25 MALE
CC110606 20 MALE
CC112141 29 MALE
CC120008 26 MALE
CC120049 28 MALE
CC120061 19 MALE
CC120065 25 FEMALE
CC120120 25 MALE
CC120123 19 FEMALE
CC120166 28 MALE
CC120182 26 MALE
CC120208 24 FEMALE
CC120218 27 FEMALE
CC120234 24 FEMALE
CC120264 28 MALE
CC120276 23 FEMALE
CC120286 22 MALE
CC120309 27 MALE
CC120313 28 MALE
CC120319 27 FEMALE
CC120347 21 FEMALE
CC120376 18 FEMALE
CC120409 18 MALE
CC120462 18 FEMALE
CC120469 25 FEMALE
CC120470 28 FEMALE
CC120550 19 MALE
CC120640 26 FEMALE
CC120727 23 FEMALE
CC120764 27 MALE
CC120795 24 MALE
CC120816 23 MALE
CC120987 20 FEMALE
CC121106 28 FEMALE
CC121111 18 MALE
CC121144 26 MALE
CC121158 28 FEMALE
CC121194 24 FEMALE
CC121200 26 FEMALE
CC121317 25 FEMALE
CC121397 27 MALE
CC12141

KeyboardInterrupt: 

### CNN Implementation with Keras 

In [7]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from sklearn.model_selection import train_test_split

TensorFlow version: 2.13.0


In [6]:
# Create Model
model = tf.keras.models.Sequential([

TensorFlow version: 2.13.0
